# Step 1 - Import packages

In [ ]:
import pandas as pd
import numpy as np
import caer
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Step 2 - Data Preprocessing

In [ ]:
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json 

In [ ]:
! kaggle datasets download -d ahemateja19bec1025/traffic-sign-dataset-classification

In [ ]:
# ! unzip /content/traffic-sign-dataset-classification.zip

In [ ]:
# remove file
! rm /content/traffic-sign-dataset-classification.zip

## Preprocessing

In [ ]:
df = pd.read_csv('./labels.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
label = df['ClassId'].values

In [ ]:
CLASS_NAME = label.tolist()
PATH_CLASS = []
for i in CLASS_NAME:
  PATH_CLASS.append(str(i))

del CLASS_NAME

In [ ]:
SIGN_NAME = df['Name'].values

In [ ]:
# building X data sets(independent weights)
train = caer.preprocess_from_dir('./traffic_Data/DATA/', classes=PATH_CLASS, IMG_SIZE=(80, 80), channels=1,)

In [ ]:
X, Y = caer.sep_train(train, IMG_SIZE=(80, 80))

In [ ]:
Y

In [ ]:
plt.imshow(X[0].reshape(80, 80))
plt.title(SIGN_NAME[Y[0]])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
del df
del X
del Y

# Step 3 - Building the model

In [ ]:
model = Sequential()

In [ ]:
# Adding CNN and Maxpool Layers
model.add(Conv2D(filters=128, kernel_size=(2, 2), activation='relu', input_shape=(80, 80, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

In [ ]:
model.add(Conv2D(filters=64, kernel_size=(4, 4), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

In [ ]:
model.add(Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

In [ ]:
model.add(Conv2D(filters=32, kernel_size=(4, 4), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=58, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# training the model

history = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

In [ ]:
# evaluation of the mode

loss, accuracy = model.evaluate(x_train, y_train)

In [ ]:
print('the training data set accuracy is', accuracy)

In [ ]:
# testing data sets accuracy and model prediction

y_pred = model.predict(x_test)

In [ ]:
pred = np.argmax(y_pred, axis=1)

In [ ]:
print('the accuracy of testing data score:', accuracy_score(y_test, pred))

In [ ]:
model.save('sign.pkl')